In [5]:
import pandas as pd
import numpy as np

# Carga df si no existe y crea logs requeridos
try:
    df
except NameError:
    df = pd.read_csv('output_price.csv')

for col, new in [('precio_promedio','ln_precio_promedio'), ('kilos','ln_kilos')]:
    if col not in df.columns:
        raise KeyError(f"Falta la columna '{col}' en el DataFrame")
    s = pd.to_numeric(df[col], errors='coerce')
    df[new] = np.where(s > 0, np.log(s), np.nan)

df.head()


c:\Users\SebastiánIgnacioCata\AppData\Local\Python\pythoncore-3.14-64\Lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
c:\Users\SebastiánIgnacioCata\AppData\Local\Python\pythoncore-3.14-64\Lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


,semana,zona_ventas,categoria,familia,sku,nombre_producto,precio_promedio,costo_por_kilo,kilos,venta,costo,ln_precio_promedio,ln_kilos
0,2024-10-28,A La Carta,CONSERVAS,VERDURAS,101520140,PALMITOS ESMERALDA 810 g,4074.0,3241.0,1,3300,2625,8.312381,0.000000
1,2024-10-28,A La Carta,OTROS ABARROTES,TORTILLAS DE TRIGO,104529120,TORTILLAS ZANAFOODS 28 cm 12 uds.,2978.0,2540.0,1,2799,2388,7.999007,0.000000
2,2024-10-28,A La Carta,PESCADOS Y MARISCOS,MARISCOS Y MOLUSCOS,104518890,SURTIDO MARISCOS SILVER 1kg IMPORTADO,2490.0,1806.0,2,4980,3611,7.820038,0.693147
3,2024-10-28,A La Carta,POLLO,TRUTROS,104519180,TRUTRO ENTERO INTERFOLIADO BRASIL 15 kg,2290.0,1617.0,15,34349,24257,7.736307,2.708050
4,2024-10-28,A La Carta,SALSAS Y ADEREZOS,KETCHUP,201200012,KETCHUP DON JUAN 900 g RRP,2133.0,1229.0,1,1920,1106,7.665285,0.000000


In [6]:
# Ajuste log-log por SKU: ln(Q) = alpha + beta ln(P) + e
import numpy as np, pandas as pd
required = ['sku','ln_precio_promedio','ln_kilos']
missing = [c for c in required if c not in df.columns]
if missing:
    raise KeyError(f'Faltan columnas: {missing}')
df_valid = df[required].replace([np.inf,-np.inf], np.nan).dropna()

def fit_group(g):
    x = g['ln_precio_promedio'].to_numpy(float)
    y = g['ln_kilos'].to_numpy(float)
    n = x.size
    if n < 2 or np.var(x) == 0:
        return pd.Series({'alpha': np.nan,'beta': np.nan,'n_puntos': int(n),'r2': np.nan})
    beta, alpha = np.polyfit(x, y, 1)
    r = np.corrcoef(x, y)[0, 1]
    return pd.Series({'alpha': float(alpha), 'beta': float(beta), 'n_puntos': int(n), 'r2': float(r**2)})

res = df_valid.groupby('sku', as_index=False).apply(fit_group).reset_index(drop=True)
if 'nombre_producto' in df.columns:
    res = res.merge(df[['sku','nombre_producto']].drop_duplicates('sku'), on='sku', how='left')
cols = ['sku'] + (['nombre_producto'] if 'nombre_producto' in res.columns else []) + ['n_puntos','beta','alpha','r2']
res = res[cols]
res = res.drop_duplicates()
res.to_csv('elasticidades_loglog_por_sku.csv', index=False)
res.head()


c:\Users\SebastiánIgnacioCata\AppData\Local\Python\pythoncore-3.14-64\Lib\site-packages\numpy\lib\_function_base_impl.py:3065: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
c:\Users\SebastiánIgnacioCata\AppData\Local\Python\pythoncore-3.14-64\Lib\site-packages\numpy\lib\_function_base_impl.py:3066: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
C:\Users\SebastiánIgnacioCata\AppData\Local\Temp\ipykernel_8120\2158271198.py:15: RankWarning: Polyfit may be poorly conditioned
  beta, alpha = np.polyfit(x, y, 1)
C:\Users\SebastiánIgnacioCata\AppData\Local\Temp\ipykernel_8120\2158271198.py:15: RankWarning: Polyfit may be poorly conditioned
  beta, alpha = np.polyfit(x, y, 1)
C:\Users\SebastiánIgnacioCata\AppData\Local\Temp\ipykernel_8120\2158271198.py:15: RankWarning: Polyfit may be poorly conditioned
  beta, alpha = np.polyfit(x, y, 1)
C:\Users\SebastiánIgnacioCata\AppData\Local\Temp\ipykernel_8120\2158271198.py:15: RankWarning: Polyfi

,sku,nombre_producto,n_puntos,beta,alpha,r2
0,40002139,TAMPER MANUAL DE ALUMINIO,4.0,0.190401,0.517639,0.010202
1,40002155,MAQUINA CAPSULA LB 2600 LAVAZZA,4.0,9.084052,-93.503664,0.966398
2,40002156,MAQUINA CAFE CAPS LB 4724 LAVAZZA,1.0,NaN,NaN,NaN
3,40002157,MAQUINA CAFE CAPS LB 2317 LAVAZZA,4.0,0.354687,-1.432320,0.316176
4,40002159,MAQUINA CAFE CAPS LB CLASSY MINI LAVAZZA,9.0,-1.972147,23.461383,0.197042


In [7]:
# Plot con botón: escribe/busca SKU, plotea ln(P) vs ln(Q) y recta del ajuste
import pandas as pd, numpy as np
import matplotlib.pyplot as plt
import ipywidgets as W
from IPython.display import display

def _ensure_df_res():
    global df, res
    if 'df' not in globals():
        df = pd.read_csv('output_price.csv')
    if 'ln_precio_promedio' not in df.columns:
        df['ln_precio_promedio'] = np.where(df['precio_promedio'] > 0, np.log(df['precio_promedio']), np.nan)
    if 'ln_kilos' not in df.columns:
        df['ln_kilos'] = np.where(df['kilos'] > 0, np.log(df['kilos']), np.nan)
    if 'res' not in globals():
        try:
            res = pd.read_csv('elasticidades_loglog_por_sku.csv')
            res = res.drop_duplicates()
        except FileNotFoundError:
            res = pd.DataFrame(columns=['sku','alpha','beta','r2','n_puntos'])

_ensure_df_res()

def _labels_map():
    skus = sorted(df['sku'].astype(str).unique().tolist())
    labels = []
    for s in skus:
        if 'nombre_producto' in df.columns:
            nm = df.loc[df['sku'].astype(str) == s, 'nombre_producto']
            lab = f"{s} - {nm.iloc[0]}" if not nm.empty else s
        else:
            lab = s
        labels.append(lab)
    return labels, skus

def _resolve(texto):
    labels, skus = _labels_map()
    if not texto:
        return None
    t = str(texto).strip()
    if t in skus:
        return t
    if ' - ' in t and t.split(' - ', 1)[0] in skus:
        return t.split(' - ', 1)[0]
    tt = t.lower()
    for lab in labels:
        if tt and tt in lab.lower():
            return lab.split(' - ', 1)[0]
    return None

def _plot(sk):
    datos = df[df['sku'].astype(str) == sk][['ln_precio_promedio','ln_kilos']].replace([np.inf, -np.inf], np.nan).dropna()
    fila = res[res['sku'].astype(str) == sk]
    alpha = float(fila['alpha'].iloc[0]) if not fila.empty else np.nan
    beta  = float(fila['beta'].iloc[0])  if not fila.empty else np.nan
    r2    = float(fila['r2'].iloc[0])    if not fila.empty else np.nan

    plt.figure(figsize=(7,5))
    x = datos['ln_precio_promedio'].to_numpy(); y = datos['ln_kilos'].to_numpy()
    plt.scatter(x, y, alpha=0.75, label='Datos (ln P vs ln Q)')
    if x.size > 0 and not (np.isnan(alpha) or np.isnan(beta)):
        xx = np.linspace(np.nanmin(x), np.nanmax(x), 100)
        yy = alpha + beta * xx
        plt.plot(xx, yy, color='red', label=f'Recta: y = {alpha:.2f} + {beta:.2f} x')
    plt.xlabel('ln(P) precio_promedio'); plt.ylabel('ln(Q) kilos')
    if not np.isnan(r2):
        xmin, xmax = plt.xlim(); ymin, ymax = plt.ylim()
        plt.text(xmin + 0.02*(xmax-xmin), ymax - 0.08*(ymax-ymin), f'R^2 = {r2:.3f}', fontsize=11, bbox=dict(facecolor='white', alpha=0.7, edgecolor='none'))
    plt.grid(True, alpha=0.25); plt.legend(); plt.show()

labels, _ = _labels_map()
cb = W.Combobox(options=labels, placeholder='Escribe SKU o nombre...', description='SKU:', ensure_option=False, layout=W.Layout(width='70%'))
btn = W.Button(description='Generar', button_style='primary', layout=W.Layout(width='20%'))
out = W.Output()

def _on_click(_):
    btn.description = 'Generando...'; btn.disabled = True
    try:
        with out:
            out.clear_output(wait=True)
            sk = _resolve(cb.value)
            if sk is None:
                print('Selecciona un SKU válido.')
            else:
                _plot(sk)
    finally:
        btn.description = 'Generar'; btn.disabled = False

btn.on_click(_on_click)
display(W.VBox([W.HBox([cb, btn]), out]))


In [ ]:
# Precio objetivo con botón: muestra últimos 10 por fecha y estadísticos en parámetros
import pandas as pd, numpy as np
import ipywidgets as W
from IPython.display import display

# Reutiliza funciones auxiliares si existen; si no, define mínimas
def _labels_map2():
    skus = sorted(df['sku'].astype(str).unique().tolist())
    labels = []
    for s in skus:
        if 'nombre_producto' in df.columns:
            nm = df.loc[df['sku'].astype(str) == s, 'nombre_producto']
            lab = f"{s} - {nm.iloc[0]}" if not nm.empty else s
        else:
            lab = s
        labels.append(lab)
    return labels, skus

def _resolve2(texto):
    labels, skus = _labels_map2()
    if not texto:
        return None
    t = str(texto).strip()
    if t in skus:
        return t
    if ' - ' in t and t.split(' - ', 1)[0] in skus:
        return t.split(' - ', 1)[0]
    tt = t.lower()
    for lab in labels:
        if tt and tt in lab.lower():
            return lab.split(' - ', 1)[0]
    return None

def _stats_for_sku(sk):
    d = df[df['sku'].astype(str) == sk]
    def s(col):
        if col not in d.columns: return (np.nan, np.nan, np.nan)
        vals = pd.to_numeric(d[col], errors='coerce')
        m = float(vals.mean()) if not np.isnan(vals.mean()) else np.nan
        st = float(vals.std(ddof=1)) if not np.isnan(vals.std(ddof=1)) else np.nan
        cv = float(st/m) if (m not in (0, np.nan) and not np.isnan(m) and not np.isnan(st)) else np.nan
        return (m, st, cv)
    pm, ps, pcv = s('precio_promedio')
    qm, qs, qcv = s('kilos')
    return pm, ps, pcv, qm, qs, qcv

labels, skus = _labels_map2()
cb2 = W.Combobox(options=labels, placeholder='Escribe SKU o nombre...', description='SKU:', ensure_option=False, layout=W.Layout(width='60%'))
vol = W.FloatText(value=10.0, description='Volumen Q:', layout=W.Layout(width='20%'))
btn2 = W.Button(description='Calcular', button_style='primary', layout=W.Layout(width='15%'))
out_tbl = W.Output(); out_params = W.Output(); out_calc = W.Output()

def _on_click2(_):
    btn2.description = 'Calculando...'; btn2.disabled = True
    try:
        sk = _resolve2(cb2.value)
        # Historial: últimos 10 por fecha
        with out_tbl:
            out_tbl.clear_output(wait=True)
            if not sk:
                print('Selecciona un SKU válido.')
            else:
                d = df[df['sku'].astype(str) == sk][['semana','precio_promedio','kilos']].copy()
                if 'semana' in d.columns:
                    with pd.option_context('mode.chained_assignment', None):
                        try:
                            d['semana'] = pd.to_datetime(d['semana'])
                        except Exception:
                            pass
                    d = d.sort_values('semana').tail(10).reset_index(drop=True)
                display(d)
        # Parámetros + estadísticos
        with out_params:
            out_params.clear_output(wait=True)
            fila = res[res['sku'].astype(str) == sk]
            if fila.empty:
                display(pd.DataFrame([{'sku': sk, 'n_puntos': 0, 'alpha': np.nan, 'beta': np.nan, 'r2': np.nan}]))
            else:
                pm, ps, pcv, qm, qs, qcv = _stats_for_sku(sk)
                base = fila[['sku','n_puntos','alpha','beta','r2']].copy().reset_index(drop=True)
                base['precio_mean'] = pm; base['precio_std'] = ps; base['precio_cv'] = pcv
                base['kilos_mean'] = qm; base['kilos_std'] = qs; base['kilos_cv'] = qcv
                display(base)
        # Precio sugerido
        with out_calc:
            out_calc.clear_output(wait=True)
            fila = res[res['sku'].astype(str) == sk]
            if fila.empty:
                print('No hay parámetros para ese SKU.')
            else:
                beta = float(fila['beta'].iloc[0]); alpha = float(fila['alpha'].iloc[0])
                if np.isnan(beta) or beta == 0 or vol.value is None or vol.value <= 0:
                    print('No es posible calcular precio (beta nula o volumen inválido).')
                else:
                    lnP = (np.log(vol.value) - alpha) / beta
                    precio = float(np.exp(lnP))
                    display(pd.DataFrame([{'sku': sk, 'Q_objetivo': vol.value, 'precio_sugerido': precio}]))
    finally:
        btn2.description = 'Calcular'; btn2.disabled = False

# Evita handlers duplicados si la celda se ejecuta varias veces
try:
    btn2._click_handlers.callbacks = []
except Exception:
    pass
btn2.on_click(_on_click2)
display(W.VBox([
    W.HBox([cb2, vol, btn2]),
    W.HTML('<b>Historial (últimos 10 por fecha)</b>'), out_tbl,
    W.HTML('<b>Parámetros + estadísticos</b>'), out_params,
    W.HTML('<b>Precio para Q objetivo</b>'), out_calc
]))
